In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras import layers, models
from sklearn.metrics import classification_report

2023-09-28 17:41:47.693558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_data():
    train_data = "Data/train"
    val_data = "Data/valid"
    return train_data, val_data

In [3]:
image_size = 224
batch_size = 50
def get_preprocessed_data(train_data, val_data):
    image_size = 224
    batch_size = 20
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255,
        shear_range= 0.2,
        horizontal_flip = True
    )
    test_val_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    train_generator = train_datagen.flow_from_directory(
        directory = train_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=True
    )
    val_generator = test_val_datagen.flow_from_directory(
        directory = val_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=False
    )
    return train_generator, val_generator

In [4]:
train_data, val_data = get_data()
train_ds, val_ds = get_preprocessed_data(train_data, val_data)

Found 10326 images belonging to 4 classes.
Found 2875 images belonging to 4 classes.


In [5]:
net = keras.applications.mobilenet_v2.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=(image_size, image_size, 3)
    )
net.summary()   
for layer in net.layers[:-2]:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(net.output)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
prediction = layers.Dense(4, activation="softmax")(x)
model = models.Model(inputs = net.input, outputs = prediction)
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [6]:
model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(learning_rate=0.001),
        metrics = ['acc']
)

In [7]:
history = model.fit(
        train_ds,
        epochs = 8,
        validation_data=val_ds
)

Epoch 1/8
517/517 [==============================] - 411s 788ms/step - loss: 0.1813 - acc: 0.9324 - val_loss: 0.0579 - val_acc: 0.9760
Epoch 2/8
517/517 [==============================] - 394s 762ms/step - loss: 0.0853 - acc: 0.9709 - val_loss: 0.0319 - val_acc: 0.9882
Epoch 3/8
517/517 [==============================] - 392s 758ms/step - loss: 0.0723 - acc: 0.9732 - val_loss: 0.0355 - val_acc: 0.9882
Epoch 4/8
517/517 [==============================] - 393s 759ms/step - loss: 0.0601 - acc: 0.9773 - val_loss: 0.0488 - val_acc: 0.9830
Epoch 5/8
517/517 [==============================] - 394s 762ms/step - loss: 0.0555 - acc: 0.9795 - val_loss: 0.0208 - val_acc: 0.9913
Epoch 6/8
517/517 [==============================] - 395s 764ms/step - loss: 0.0572 - acc: 0.9785 - val_loss: 0.0152 - val_acc: 0.9941
Epoch 7/8
517/517 [==============================] - 399s 772ms/step - loss: 0.0426 - acc: 0.9834 - val_loss: 0.0328 - val_acc: 0.9889
Epoch 8/8
517/517 [==============================] - 39

In [8]:
scores = model.evaluate(val_ds, verbose=1)
print(f"Loss: {scores[0]}, Accuracy : {scores[1]}")

144/144 [==============================] - 78s 544ms/step - loss: 0.0168 - acc: 0.9941
Loss: 0.01678372733294964, Accuracy : 0.9940869808197021


In [ ]:
test_pred = model.predict(val_ds, verbose = 1)
test_labels = np.argmax(test_pred, axis=1)

 95/144 [==================>...........] - ETA: 26s

In [ ]:
class_labels = val_ds.class_indices
class_labels = {v:k for k,v in class_labels.items()}
classes = list(class_labels.values())
print(classes)
print('Classification Report')
print(classification_report(val_ds.classes, test_labels, target_names=classes))

In [ ]:
model.save("model/GrapeLeafMobileNet.h5")